In [1]:
import json
import torch.nn.functional as F
from model import *
from tokenizer import *
import logging
import sys
from typing import Callable, Dict, List, NoReturn, Tuple
import torch
import numpy as np
from transformers import AutoTokenizer, set_seed

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
set_seed(42)

In [37]:
with open("../../json_data/blogs_data/blogs_data.json", "r", encoding="utf-8") as f:
        wiki = json.load(f)
context = list(dict.fromkeys([v["content"] for v in wiki.values()]))
urls = list(dict.fromkeys([v["url"] for v in wiki.values()]))

In [29]:
MODEL_NAME = "klue/bert-base"
tokenizer = load_tokenizer(MODEL_NAME)
model = ColbertModel.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(tokenizer.vocab_size + 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.load_state_dict(torch.load(f"./blog_model/fine_tuned_blog_colbert.pth"))

Some weights of the model checkpoint at klue/bert-base were not used when initializing ColbertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing ColbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColbertModel were not initialized from the model checkpoint at klue/bert-base and are newly initializ

<All keys matched successfully>

In [30]:
query = "제주도에 고기국수가 맛있는데 있는지 찾아봐야 할 것 같다. 제주도 흑돼지는 숙성도 좋고 말고기는 마우돈이 괜찮다고 해서 먹어보고 싶다고 이야기한다."
# 고기국수, 흑돼지 , 말고기 

In [31]:
def preprocess(text):
    text = re.sub(r"[^A-Za-z0-9가-힣.?!,()~‘’“”"":&<>·\-\'+\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()  # 두 개 이상의 연속된 공백을 하나로 치환
    return text  

In [32]:
def tokenize(dataset, tokenizer):
    preprocessed_data="[Q] " + preprocess(query)
    tokenized_query = tokenizer(
        preprocessed_data, return_tensors="pt", padding=True, truncation=True, max_length=128
    )
    return tokenized_query

In [33]:
def get_score(Q, D, N=None, eval=False):
        # hard negative N은 train에만 쓰임.
        if eval:
            final_score = torch.tensor([])
            for D_batch in tqdm(D):
                D_batch = np.array(D_batch)
                D_batch = torch.Tensor(D_batch).squeeze()
                print("Docu_dim_size!! : ",D_batch.shape)
                p_seqeunce_output = D_batch.transpose(
                    1, 2
                )  # (batch_size,hidden_size,p_sequence_length)-> 200 128 512
                print("Query_dim_size!! : ",Q.size()) 
                q_sequence_output = Q.view(
                    1, 1, -1, 128
                )  # (1, 1, q_sequence_length, hidden_size)
                dot_prod = torch.matmul(
                    q_sequence_output, p_seqeunce_output
                )  # (1,batch_size, q_sequence_length, p_seqence_length)
                max_dot_prod_score = torch.max(dot_prod, dim=3)[
                    0
                ]  # (batch_size,batch_size,q_sequnce_length)
                score = torch.sum(max_dot_prod_score, dim=2)  # (batch_size,batch_size)
                final_score = torch.cat([final_score, score], dim=1)
            print("final_score!! :",final_score.size())
            return final_score

In [35]:
batched_p_embs = []

with torch.no_grad():

    model.eval()

    q_seqs_val = tokenize(query, tokenizer).to("cuda")
    q_emb = model.query(**q_seqs_val).to("cpu")

    print("Start passage embedding......")
    p_embs = []
    for step, p in enumerate(tqdm(context)):
        p = tokenize_colbert(p, tokenizer, corpus="doc").to("cuda")
        p_emb = model.doc(**p).to("cpu").numpy()
        p_embs.append(p_emb)
        if (step + 1) % 200 == 0:
            batched_p_embs.append(p_embs)
            p_embs = []
    batched_p_embs.append(p_embs)

dot_prod_scores = get_score(q_emb, batched_p_embs, eval=True)

print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print("score !! : ",dot_prod_scores)
print("rank and rank_size!! :",rank)
print(rank.size())


Start passage embedding......


 11%|█         | 3/27 [00:00<00:00, 29.69it/s]

Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])


 33%|███▎      | 9/27 [00:00<00:00, 23.77it/s]

Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])


...

Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])


 78%|███████▊  | 21/27 [00:00<00:00, 24.51it/s]

Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])


100%|██████████| 27/27 [00:01<00:00, 24.57it/s]

Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([200, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
Docu_dim_size!! :  torch.Size([121, 512, 128])
Query_dim_size!! :  torch.Size([1, 45, 128])
final_score!! : torch.Size([1, 5321])
torch.Size([1, 5321])
score !! :  tensor([[28.4640, 29.3031, 28.4482,  ..., 28.9409, 27.9027, 29.3155]])
rank and rank_size!! : tensor([4810, 4214, 4761,  ..., 3797, 2268,  727])
torch.Size([5321])


In [36]:
k = 5

for i in range(k):
    print(f"TOP {i+1} : \n",context[rank[i]])
    print("="*40)

TOP 1 : 
 음식에 진심이 무엇을 말하는지 깊이 생각해 본 적도 없고 그러고 싶지도 않지만 입에 맞는 음식을 먹는다는 것은 여행에 있어 아주 중요한 요소다. 더군다나 나 혼자만의 여행이 아닌 누군가와 만남을 갖는 때라면 먹는 것이 몹시 중요해지기는 순간이다. 삼각형 모양의 주차장이 야자수 나무와 어울려 꽤 낭만적으로 보이는 짚불도 애월점에 도착. 전에 들렀었을 때 2층은 치킨집이라고 들었던 것 같고 1층이 제주 애월 흑돼지 맛집으로 이미 소문이 나버린 짚불도 애월점이다. 오늘은 이곳에서 선배 부부와 만나기로 했고 선배 내외를 모두 만나는 건 수년 만이라 내가 아는 선상에서 최대한 맛이 좋았던 곳에서 식사를 하고 싶었다. 짚불도는 그 맛과 향이 독특해 인기가 높아진 듯하며 초벌로 짚불을 이용해 굽고 테이블에서 고기를 구울 때 다시 한번 짚불로 훈현을 하기 때문에 잡내가 없고 짚불의 향이 그대로 남아있어 특별하게 느껴진다. 실내는 그리 크지 않다.
사각보다 활용성이 높다는 원형 테이블이며 의자는 하단부에 짐을 넣을 수 있도록 해 공간 활용을 최대한 이끌어 낸 듯한 인테리어가 특징이라면 특징일 수 있겠다. 최초 의자에 앉으면 기본 찬을 서비스해준다. 이후에 발생하는 추가 반찬 요구는 스스로 해결해야 하는 셀프 코너를 운영하고 있어 반찬이 떨어졌다 하여 주인장을 부르고 기다리는 일 없이 즉시 가져오면 된다. 어디서나 반납 가능한 충전 돼지 보조 배터리 대여.
세상은 갈수록 좋아지는 것 같다.
제주 애월 흑돼지 맛집에서 보조 배터리를 대여할 수 있을 거라 누가 생각이나 했겠는가?
금액이 얼마인지 모르겠지만 마침 보조 배터리를 챙기지 못해 스마트폰이 꼴까닥 꼴까닥 하는 중이라면 가격에 대한 심리적 저항선이 매우 높을 수 있단 생각이 든다. 기본찬이 다 깔렸다.
뭐 이런 풀때기?라고 생각하지 마시길.
풀때기 자체로 먹어야 하는 건 그 자체로 맛이 좋고 필히 불판 위에 올려 드시라 말하고 싶은 것이 2가지 있다.
그 하나는 고사리로 불판에서 살짝 구우며 애월 흑돼지가 

In [38]:
for i in range(k):
    print(f"TOP {i+1} : \n",urls[rank[i]])
    print("="*40)
"""
query = "제주도에 고기국수가 맛있는데 있는지 찾아봐야 할 것 같다. 제주도 흑돼지는 숙성도 좋고 말고기는 마우돈이 괜찮다고 해서 먹어보고 싶다고 이야기한다."
# 고기국수, 흑돼지 , 말고기 
"""

TOP 1 : 
 https://blog.naver.com/kooni/222958189767
TOP 2 : 
 https://blog.naver.com/hloveh01/222969736779
TOP 3 : 
 https://blog.naver.com/danzzak78/222951432070
TOP 4 : 
 https://blog.naver.com/rose0626/222945456032
TOP 5 : 
 https://blog.naver.com/o0owldus/222983067494
